In [10]:
import csv
import emoji
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

In [11]:
def read_csv(filename):
    phrase = []
    emoji = []

    with open(filename) as emoji_data:
        csvReader = csv.reader(emoji_data)
        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)
    return X, Y

In [12]:
X_train, Y_train = read_csv('./dataset/train_emoji.csv')
X_test, Y_test = read_csv('./dataset/test_emoji.csv')

In [36]:
emoji_dictionary = {
    "0": ":heart_with_arrow:",
    "1": ":baseball:",
    "2": ":smile:",
    "3": ":disappointed:",
    "4": ":fork_and_knife:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[str(label)],  language='alias')

In [38]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much 💘
food is life 🍴
I love you mum 💘
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾
